In [ ]:
import numpy as np # работа с датасетом
import pandas as pd # математическая библиотека
from pandas import read_csv, DataFrame, Series # чтение данных
from sklearn.model_selection import train_test_split, KFold, cross_val_score # подготовка дынных и анализ результатов
import matplotlib.pyplot as plt # построение графика
from sklearn.preprocessing import LabelEncoder # манипуляции с данными

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

Для начала считаем данные из файла в переменную data, чтобы в дальнейшем мы смогли свободно с ними работать

In [ ]:
data = read_csv('../input/german-credit-data-with-risk/german_credit_data_with_risk.csv')

Узнаем размеры таблицы:

In [ ]:
data.shape

Итак, таблица содержит 1000 строк (объектов) и 11 столбцов (признаков), включая выходной (целевой) признак.

Мы можем посмотреть на несколько первых и несколько последних строк этой таблицы, чтобы получить представление об имеющихся данных:

In [ ]:
data.head(3)

In [ ]:
data.tail(3)

Похоже, что первый столбец - это просто индекс, который мы можем удалить.

In [ ]:
data.drop(data.columns[0], inplace=True, axis=1)

In [ ]:
data.head(3)

In [ ]:
data.tail(3)

Получим некоторую сводную информацию по всей таблице. По умолчанию будет выдана информация только для количественных признаков. Это общее их количество (count), среднее значение (mean), стандартное отклонение (std), минимальное (min), макcимальное (max) значения, медиана (50%) и значения нижнего (25%) и верхнего (75%) квартилей:

In [ ]:
data.describe()

Видим,что нет пропущенных значений у количественных признаков

Выделим числовые и категориальные признаки:

In [ ]:
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
numerical_columns   = [c for c in data.columns if data[c].dtype.name != 'object']
print(categorical_columns)
print(numerical_columns)

Теперь мы можем получить некоторую общую информацию по категориальным признакам:

In [ ]:
data[categorical_columns].describe()

Определим полный перечень значений категориальных признаков:

In [ ]:
for c in categorical_columns:
    print(data[c].unique())

In [ ]:
from pandas.tools.plotting import scatter_matrix
scatter_matrix(data, alpha=0.05, figsize=(20, 20));

Посмотрим корреляцию количественных признаков:

In [ ]:
data.corr()

Видим, что практиччески все признаки не сильно коррелируют, т.к. по модулю значения не превышают 0,3. Но есть признак Credit amount, где корреляция уже достаточно ощутимая

Узнаем количество заполненных элементов. Параметр axis = 0 указывает, что мы двигаемся по размерности 0 (сверху вниз), а не размерности 1 (слева направо), т.е. нас интересует количество заполненных элементов в каждом столбце, а не строке:

In [ ]:
data.count(axis=0)

Если данные имеют пропущенные значения, существует несколько альтернатив. Удаление строк с пропущенными, удаление столбцов с пропущенными значениями. Но тогда данных станет горазо меньше. К примеру, если мы удалим строки со всеми пропущенными значениями стобца Checking account, то количество данных сократится 396 строк. А это уже весомая потеря наших данных. 

Поэтому мы рассмотрим два альтернативных способа. 
1. Замена пропущенных значений на самое популярное в столбце.
2. Замена пропущенных значений новым признаком. Например, no_inf

Данные для первого способа мы сохраним в переменную data_top, а для второго способа в переменную data_new. И в итоге посмотрим, какой способ лучше.

In [ ]:
data_top = read_csv('../input/german-credit-data-with-risk/german_credit_data_with_risk.csv')
data_new = read_csv('../input/german-credit-data-with-risk/german_credit_data_with_risk.csv')

Для варианта с самыми популярными значениями заполним неопределенные признаки самым популярным значением:

In [ ]:
data_describe = data_top.describe(include=[object])
for c in categorical_columns:
    data_top[c] = data_top[c].fillna(data_describe[c]['top'])

In [ ]:
data_top.head(10)

Для варианта с новым значением введем значение no_inf и заполним им все пропущенные:

In [ ]:
data_describe = data_new.describe(include=[object])
for c in categorical_columns:
    data_new[c] = data_new[c].fillna('no_inf')

In [ ]:
data_new.head(10)

Библиотека scikit-learn не умеет напрямую обрабатывать категориальные признаки. Поэтому прежде чем подавать данные на вход алгоритмов машинного обучения преобразуем категориальные признаки в количественные.

Категориальные признаки, принимающие два значения (т.е. бинарные признаки) и принимающие большее количество значений будем обрабатывать по-разному.

Вначале выделим бинарные и небинарные признаки:

In [ ]:
binary_columns    = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]
print('Binary:', binary_columns)
print('Non binary:', nonbinary_columns)

Значения бинарных признаков просто заменим на 0 и 1. Чтобы избежать дублирования кода, напишем простую функцию для кодирования столбцов.

In [ ]:
def SetBinary(data):
    label = LabelEncoder()
    dicts = {}

    label.fit(data.Sex.drop_duplicates())
    dicts['Sex'] = list(label.classes_)
    data.Sex = label.transform(data.Sex)

    label.fit(data.Risk.drop_duplicates())
    dicts['Risk'] = list(label.classes_)
    data.Risk = label.transform(data.Risk)
    return data

In [ ]:
data_new = SetBinary(data_new)
dara_top = SetBinary(data_top)

Просмотрим, все ли у нас успешно получилось:

In [ ]:
data_top.head(5)

In [ ]:
data_new.head(5)

Далее у нас остались небинарные признаки. К небинарными признакам применим метод векторизации.

In [ ]:
data_nonbinary_top = pd.get_dummies(data_top[nonbinary_columns])
print(data_nonbinary_top.columns)

In [ ]:
data_nonbinary_new = pd.get_dummies(data_new[nonbinary_columns])
print(data_nonbinary_new.columns)

Многие алгоритмы машинного обучения чувствительны к масштабированию данных. К таким алгоритмам, например, относится метод ближайших соседей, машина опорных векторов и др.

В этом случае количественные признаки полезно нормализовать. Это можно делать разными способами. Например, каждый количественный признак приведем к нулевому среднему и единичному среднеквадратичному отклонению:

In [ ]:
data_numerical_top = data_top[numerical_columns]
data_numerical_top = (data_numerical_top - data_numerical_top.mean()) / data_numerical_top.std()
data_numerical_top.describe()

In [ ]:
data_numerical_new = data_new[numerical_columns]
data_numerical_new = (data_numerical_new - data_numerical_new.mean()) / data_numerical_new.std()
data_numerical_new.describe()

Соединим все столбцы в одну таблицу:

In [ ]:
data_top = pd.concat((data_numerical_top, data_top[binary_columns], data_nonbinary_top), axis=1)
data_top = pd.DataFrame(data_top, dtype=float)
data_top.head(5)

In [ ]:
data_new = pd.concat((data_numerical_new, data_new[binary_columns], data_nonbinary_new), axis=1)
data_new = pd.DataFrame(data_new, dtype=float)
data_new.head(5)

Посмотрим размеры получившихся таблиц

In [ ]:
data_top.shape

In [ ]:
data_new.shape

Разделим входные признаки и выделенные призак:

In [ ]:
X_top = data_top.drop(('Risk'), axis=1)
y_top = data_top['Risk']

X_new = data_new.drop(('Risk'), axis=1)
y_new = data_new['Risk']

Обучаться, или, как говорят, строить модель, мы будем на обучающей выборке, а проверять качество построенной модели – на тестовой. В соревнованиях и конкурсах по анализу данных и машинному обучению участнику доступна только обучающая выборка, а тестовая неизвестна.

В рассматриваемой задаче мы сами разобьем имеющиеся у нас данные на обучающую и тестовую выборки (на самом деле, это больше соответствует реальной ситуации, с которой сталкиваются исследователи).

Разбиение на тестовую и обучающую выборку должно быть случайным. Обычно используют разбиения в пропорции 50%:50%, 60%:40%, 75%:25% и т.д.

Мы воспользуемся функцией train_test_split из модуля sklearn.cross_validation. и разобьем данные на обучающую/тестовую выборки в отношении 70%:30%:

In [ ]:
X_train_top, X_test_top, y_train_top, y_test_top = train_test_split(X_top, y_top, test_size = 0.3, random_state = 11)
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size = 0.3, random_state = 11)

Заранее заведем список, где будут храниться результаты "предсказаний" по тестовой выбрке методов, которые мы будем использовать

In [ ]:
itog_val = {} #список для записи результатов работы методов

Начнем с одного из самых простых алгоритмов машинного обучения – метода k ближайших соседей (kNN).

Для нового объекта алгоритм ищет в обучающей выборке k наиболее близких объекта и относит новый объект к тому классу, которому принадлежит большинство из них.

Количество соседей k соответствует параметру n_neighbors. По умолчанию, n_neighbors = 5.

Вначале обучим модель:

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train_top, y_train_top)

После того, как модель обучена, мы можем предсказывать значение целевого признака по входным признакам для новых объектов. Делается это с помощью метода predict.

Нас интересует качество построенной модели, поэтому будем предсказывать значение выходного признака на тех данных, для которых оно известно: на обучающей и (что более важно) тестовой выборках:

In [ ]:
y_train_predict_top = knn.predict(X_train_top)
y_test_predict_top = knn.predict(X_test_top)

err_train = np.mean(y_train_top != y_train_predict_top)
err_test  = np.mean(y_test_top  != y_test_predict_top)
print('Train top error:', err_train)
print('Test top error', err_test)
itog_val['KNeighborsClassifierTop'] = np.mean(y_test_top == y_test_predict_top)

err_train и err_test – это ошибки на обучающей и тестовой выборках. Как мы видим, они составили 23.3% и 34.7%.

Для нас более важным является ошибка на тестовой выборке, так как мы должны уметь предсказывать правильное (по возможности) значение на новых объектах, которые при обучении были недоступны.

Мы обучили модель и предсказали на данных, в которых пропущенные значения мы заполнили популярными значениями. Теперь давате проделаем тоже самое уже на других данных - с новым значениями.

In [ ]:
knn.fit(X_train_new, y_train_new)

In [ ]:
y_train_predict_new = knn.predict(X_train_new)
y_test_predict_new = knn.predict(X_test_new)

err_train = np.mean(y_train_new != y_train_predict_new)
err_test  = np.mean(y_test_new  != y_test_predict_new)
print('Train new error:', err_train)
print('Test new error', err_test)

itog_val['KNeighborsClassifierNew'] = np.mean(y_test_new == y_test_predict_new)

Хм, заполнение пустых значений новыми признаками в методе k соседей дает небольшое, но уменьшение ошибки. Ошибка в тестовой выборке при замене на наиболее популярное дало 34,7% ошибки, в то время как при добавлении нового признака дает ошибку в 32,3%. Преимущество конечно не существенное, но все же чем меньше - тем лучше :)

Давайте проверим данную тенденцию еще на двух алгоритмах и уже окончательно определимся с итоговыми данными.

Следующий метод, который мы попробуем – машина опорных векторов (SVM – support vector machine или SVC – support vector classifier) – сразу приводит к более оптимистичным результатам.

Уже со значением параметров по умолчанию (в частности, ядро – радиальное rbf) получаем более низкую ошибку на обучающей выборке:

In [ ]:
svc = SVC()
svc.fit(X_train_top, y_train_top)

err_train = np.mean(y_train_top != svc.predict(X_train_top))
err_test  = np.mean(y_test_top  != svc.predict(X_test_top))
print('Train top error:', err_train)
print('Test top error', err_test)

itog_val['SVC_Top'] = np.mean(y_test_top == y_test_predict_top)

Итак, на тестовой выборке, где пропущенные значения были заполнены популярными, получили ошибку в 32%. Теперь проверим данный алгоритм на других данных, где мы заполнили новыми значениями пропущеннные

In [ ]:
svc.fit(X_train_new, y_train_new)
y_train_predict_new = svc.predict(X_train_new)
y_test_predict_new = svc.predict(X_test_new)

err_train = np.mean(y_train_new != y_train_predict_new)
err_test  = np.mean(y_test_new  != y_test_predict_new)
print('Train new error:', err_train)
print('Test new error', err_test)

itog_val['SVC_New'] = np.mean(y_test_new == y_test_predict_new)

In [ ]:
rf = RandomForestClassifier()

rf.fit(X_train_top, y_train_top)

err_train = np.mean(y_train_top != rf.predict(X_train_top))
err_test  = np.mean(y_test_top  != rf.predict(X_test_top))
print('Train top error:', err_train)
print('Test top error', err_test)

itog_val['RandomForest_Top'] = np.mean(y_test_top == y_test_predict_top)

In [ ]:
rf.fit(X_train_new, y_train_new)
y_train_predict_new = rf.predict(X_train_new)
y_test_predict_new = rf.predict(X_test_new)

err_train = np.mean(y_train_new != y_train_predict_new)
err_test  = np.mean(y_test_new  != y_test_predict_new)
print('Train new error:', err_train)
print('Test new error', err_test)

itog_val['RandomForest_New'] = np.mean(y_test_new == y_test_predict_new)

In [ ]:
DataFrame.from_dict(data = itog_val, orient='index').plot(kind='bar', legend=False, figsize=(10,6))